In [23]:
import nfl_data_py as nfl
import pandas as pd
import sqlite3 as sq
pd.set_option('display.max_columns',30)

In [24]:
conn = sq.connect('data/nfl.db')

In [25]:
end_year = 2026

years=range(2000,end_year)

In [ ]:
weekly = nfl.import_weekly_data(range(2000,end_yearselect sel;))
weekly.to_parquet('data/weekly.parquet')

HTTPError: HTTP Error 404: Not Found

In [ ]:
weekly.to_sql('weekly', conn, if_exists='replace', index=False)

129439

In [ ]:
conn.execute('create index week_index on weekly(week)')
conn.execute('create index season_index on weekly(season)')

In [ ]:
query = 'select * from weekly where position = "RB" and recent_team = "IND"'
result_df = pd.read_sql_query(query, conn)
result_df

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,...,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0000003,None,Abdul-Karim al-Jabbar,RB,RB,None,IND,2000,1,REG,KC,0,0,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,-0.2,-0.2
1,00-0006155,None,Lennox Gordon,RB,RB,None,IND,2000,15,REG,BUF,0,0,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.6,0.6
2,00-0006155,None,Lennox Gordon,RB,RB,None,IND,2000,17,REG,MIN,0,0,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.1,0.1
3,00-0008241,None,Edgerrin James,RB,RB,None,IND,2000,1,REG,KC,0,0,0.0,0,...,1,0.0,0.0,0.0,0.0,1.0,2.409343,0,0.00,0.206897,NaN,NaN,0.0,26.1,31.1
4,00-0008241,None,Edgerrin James,RB,RB,None,IND,2000,2,REG,LV,0,0,0.0,0,...,1,0.0,0.0,0.0,0.0,2.0,1.015774,0,0.00,0.148936,NaN,NaN,0.0,24.1,30.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,00-0037120,T.Goodson,Tyler Goodson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,IND,2024,8,REG,HOU,0,0,0.0,0,...,0,0.0,0.0,25.0,0.0,0.0,-0.593228,0,0.00,0.032258,0.051125,0.084174,0.0,0.0,0.0
969,00-0037120,T.Goodson,Tyler Goodson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,IND,2024,10,REG,BUF,0,0,0.0,0,...,1,0.0,0.0,12.0,3.0,1.0,1.822438,0,0.75,0.085714,0.043478,0.159006,0.0,6.9,8.9
970,00-0037120,T.Goodson,Tyler Goodson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,IND,2024,15,REG,DEN,0,0,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.3,0.3
971,00-0037120,T.Goodson,Tyler Goodson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,IND,2024,16,REG,TEN,0,0,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,2.2,2.2


In [ ]:
pbp = nfl.import_pbp_data(years)

2000 done.
2001 done.
2002 done.
2003 done.
2004 done.
2005 done.
2006 done.


KeyboardInterrupt: 

In [ ]:
pbp.to_parquet('data/pbp.parquet')

In [ ]:
pbp.to_sql('pbp', conn, if_exists='replace', index=False)

1185053

In [ ]:
conn.execute('create index date_index on pbp(game_date)')
#conn.execute('create index season_index on pbp(season)')

In [ ]:
rosters = nfl.import_weekly_rosters(range(2002,end_year))
rosters

,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,...,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age
0,2002,CHI,RB,None,27,ACT,Rabih Abdullah,Rabih,Abdullah,1975-04-27,72.0,220.0,None,00-0000007,None,...,4.0,https://static.www.nfl.com/image/private/f_aut...,None,14,REG,A01,Rabih,ABD675101,None,32004142-4467-5101-8326-4b71f9f0a7b7,1998.0,1998.0,None,None,27.619
1,2002,CHI,RB,None,27,ACT,Rabih Abdullah,Rabih,Abdullah,1975-04-27,72.0,220.0,None,00-0000007,None,...,4.0,https://static.www.nfl.com/image/private/f_aut...,None,5,REG,A01,Rabih,ABD675101,None,32004142-4467-5101-8326-4b71f9f0a7b7,1998.0,1998.0,None,None,27.447
2,2002,CHI,RB,None,27,ACT,Rabih Abdullah,Rabih,Abdullah,1975-04-27,72.0,220.0,None,00-0000007,None,...,4.0,https://static.www.nfl.com/image/private/f_aut...,None,13,REG,A01,Rabih,ABD675101,None,32004142-4467-5101-8326-4b71f9f0a7b7,1998.0,1998.0,None,None,27.598
3,2002,CHI,RB,None,27,ACT,Rabih Abdullah,Rabih,Abdullah,1975-04-27,72.0,220.0,None,00-0000007,None,...,4.0,https://static.www.nfl.com/image/private/f_aut...,None,2,REG,A01,Rabih,ABD675101,None,32004142-4467-5101-8326-4b71f9f0a7b7,1998.0,1998.0,None,None,27.387
4,2002,CHI,RB,None,27,ACT,Rabih Abdullah,Rabih,Abdullah,1975-04-27,72.0,220.0,None,00-0000007,None,...,4.0,https://static.www.nfl.com/image/private/f_aut...,None,16,REG,A01,Rabih,ABD675101,None,32004142-4467-5101-8326-4b71f9f0a7b7,1998.0,1998.0,None,None,27.655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862571,2025,KC,LB,LB,49,CUT,Xander Mueller,Xander,Mueller,NaT,NaN,232.0,None,00-0040780,None,...,0.0,None,None,1,REG,W03,Xander,MUE622935,59264,32004d55-4562-2935-86f2-6931bc469b27,2025.0,2025.0,None,NaN,NaN
862572,2025,IND,LB,LB,49,CUT,Joseph Vaughn,Joseph,Vaughn,NaT,NaN,238.0,None,00-0040781,None,...,0.0,None,None,1,REG,W03,Joseph,VAU443667,59277,32005641-5544-3667-4846-303a16259708,2025.0,2025.0,None,NaN,NaN
862573,2025,CLE,WR,WR,16,ACT,Isaiah Bond,Isaiah,Bond,2004-03-15,71.0,180.0,Texas; Alabama,00-0040782,None,...,0.0,None,None,1,REG,A01,Isaiah,BON168854,59278,3200424f-4e16-8854-5500-4fcd95690bde,2025.0,2025.0,None,NaN,21.481
862574,2025,IND,LB,LB,47,CUT,Jake Chaney,Jake,Chaney,NaT,NaN,230.0,None,00-0040783,None,...,0.0,None,None,1,REG,W03,Jake,CHA365039,59276,32004348-4136-5039-1d25-b165494652ef,2025.0,2025.0,None,NaN,NaN


In [ ]:
rosters.to_sql('rosters', conn, if_exists='replace', index=False)

859529

In [ ]:
ids = nfl.import_ids()

In [ ]:
ids.to_sql('ids', conn, if_exists='replace', index=False)

12115

In [ ]:
snaps = nfl.import_snap_counts(range(2012,end_year))
team_map = {'OAK':'LV',
            'STL':'LA',
            'SD':'LAC'}
snaps['team'] = snaps['team'].apply(lambda x: team_map.get(x,x))
snaps.to_sql('snaps', conn, if_exists='replace', index=False)

295770

In [ ]:
depth = nfl.import_depth_charts(range(2001,end_year)).drop_duplicates().reset_index(drop=True)

team_map = {'OAK':'LV',
            'STL':'LA',
            'SD':'LAC'}
depth['club_code'] = depth['club_code'].apply(lambda x: team_map.get(x,x))

depth = depth.sort_values(by=['season','club_code','week','depth_team'])
depth['dense_depth'] = depth.groupby(['club_code','season','week','position', 'game_type','formation']).cumcount()
depth['depth_position'] = depth['depth_position'].replace({'\n    ':None})
depth['depth_position'] = depth['depth_position'].fillna(depth['position'])
depth = depth.rename({'club_code':'team_name'},axis=1)
depth.to_sql('depth', conn, if_exists='replace', index=False)

865329

In [ ]:
schedule = nfl.import_schedules(years)
#schedule.to_sql('schedule', conn, if_exists='replace', index=False)
schedule

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,...,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
259,2000_01_SF_ATL,2000,REG,1,2000-09-03,Sunday,13:00,SF,28.0,ATL,36.0,Home,8.0,64.0,0.0,...,NaN,1,dome,astroturf,NaN,NaN,00-0005755,00-0002876,Jeff Garcia,Chris Chandler,Steve Mariucci,Dan Reeves,Walt Coleman,ATL00,Georgia Dome
260,2000_01_JAX_CLE,2000,REG,1,2000-09-03,Sunday,13:00,JAX,27.0,CLE,7.0,Home,-20.0,34.0,0.0,...,NaN,1,outdoors,grass,78.0,6.0,00-0002110,00-0003535,Mark Brunell,Tim Couch,Tom Coughlin,Chris Palmer,Bernie Kukar,CLE00,Cleveland Browns Stadium
261,2000_01_IND_KC,2000,REG,1,2000-09-03,Sunday,13:00,IND,27.0,KC,14.0,Home,-13.0,41.0,0.0,...,NaN,0,outdoors,grass,90.0,5.0,00-0010346,00-0006300,Peyton Manning,Elvis Grbac,Jim Mora,Gunther Cunningham,Larry Nemmers,KAN00,Arrowhead Stadium
262,2000_01_CHI_MIN,2000,REG,1,2000-09-03,Sunday,13:00,CHI,27.0,MIN,30.0,Home,3.0,57.0,0.0,...,NaN,1,dome,astroturf,NaN,NaN,00-0011044,00-0003739,Cade McNown,Daunte Culpepper,Dick Jauron,Dennis Green,Ed Hochuli,MIN00,Hubert H. Humphrey Metrodome
263,2000_01_TB_NE,2000,REG,1,2000-09-03,Sunday,13:00,TB,21.0,NE,16.0,Home,-5.0,37.0,0.0,...,NaN,0,outdoors,grass,71.0,5.0,00-0009266,00-0001361,Shaun King,Drew Bledsoe,Tony Dungy,Bill Belichick,Jeff Triplette,BOS99,Foxboro Stadium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7258,2025_18_DAL_NYG,2025,REG,18,2026-01-04,Sunday,13:00,DAL,NaN,NYG,NaN,Home,NaN,NaN,NaN,...,NaN,1,outdoors,fieldturf,NaN,NaN,NaN,NaN,NaN,NaN,Brian Schottenheimer,Brian Daboll,NaN,NYC01,MetLife Stadium
7259,2025_18_WAS_PHI,2025,REG,18,2026-01-04,Sunday,13:00,WAS,NaN,PHI,NaN,Home,NaN,NaN,NaN,...,NaN,1,outdoors,grass,NaN,NaN,NaN,NaN,NaN,NaN,Dan Quinn,Nick Sirianni,NaN,PHI00,Lincoln Financial Field
7260,2025_18_BAL_PIT,2025,REG,18,2026-01-04,Sunday,13:00,BAL,NaN,PIT,NaN,Home,NaN,NaN,NaN,...,NaN,1,outdoors,grass,NaN,NaN,NaN,NaN,NaN,NaN,John Harbaugh,Mike Tomlin,NaN,PIT00,Acrisure Stadium
7261,2025_18_SEA_SF,2025,REG,18,2026-01-04,Sunday,13:00,SEA,NaN,SF,NaN,Home,NaN,NaN,NaN,...,-115.0,1,outdoors,grass,NaN,NaN,NaN,NaN,NaN,NaN,Mike Macdonald,Kyle Shanahan,NaN,SFO01,Levi's Stadium


In [ ]:
conn.commit()
conn.close()